# Notebook 1: Data Cleaning

... Because data is never clean!

This notebook:
 - reads raw data sources
 - cleans and conforms the mess
 - unions all of the inputs (one per year) and pickles the dataframe!

In [1]:
import pandas as pd
from glob import glob
import re
import os

In [2]:
raw_data_files = glob('../data/raw/*')
df_dict = {}
for file in raw_data_files:
    try:
        yr = re.search(r'(20\d{2})', file).group(0)
        print('reading year {}'.format(yr))
        df = pd.read_excel(file)
        df_dict[yr] = df
    except:
        if 'Wards.csv' in file:
            print('reading the wards csv')
            wards = pd.read_csv(file)
        elif 'ward_population' in file:
            print('reading the ward population csv')
            ward_population = pd.read_csv(file)



reading year 2019
reading year 2013
reading year 2016
reading the ward population csv
reading the wards csv
reading year 2017
reading year 2021
reading year 2018
reading year 2015
reading year 2014
reading year 2020


In [3]:
ward_population['ward'] = ward_population.ward.str.upper().str.replace('\d{1,2}. ', '')
wards = wards.merge(ward_population, left_on='WARD_EN', right_on='ward', how='outer')

/var/folders/dg/pbzylmz55yn_lm8754cv4dfc0000gp/T/ipykernel_3237/3941286951.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  ward_population['ward'] = ward_population.ward.str.upper().str.replace('\d{1,2}. ', '')


In [4]:
# the field names and values from 2013, 2014 and 2015 differ slightly from other years
# Here we will conform those years with 2016+
for yr in ['2013','2014','2015']:
    # pull timestamp from string
    df_dict[yr]['ts'] = pd.to_datetime(df_dict[yr].creation_date)
    # grab ward number from ward field (numeric value instead of name)
    df_dict[yr]['ward_number'] = df_dict[yr].ward.str.extract('(\d+)').fillna(0).astype(int)
    # merge with ward dataset to get ward name from the number
    df_dict[yr] = df_dict[yr].merge(wards[['WARD_NUM','WARD_EN']].rename(columns={'WARD_EN':'WARD'}), left_on='ward_number', right_on='WARD_NUM', how='left')
    df_dict[yr].rename(columns={'priority_name': 'CHANNEL', 'call_type': 'REASON', 'call_description':'TYPE', 'ts':'DATE_RAISED'}, inplace=True)
    df_dict[yr]['SUBJECT'] = df_dict[yr].REASON
    df_dict[yr] = df_dict[yr][df_dict['2020'].columns]

In [5]:
# Appears to be a UTF encoding issue with ORLÉANS, so here's a manual fix
df_dict['2019']['WARD'] = df_dict['2019'].WARD.str.replace('ORL&#201;ANS', 'ORLÉANS')
df_dict['2020']['WARD'] = df_dict['2020'].WARD.str.replace('ORL&#201;ANS', 'ORLÉANS')
df_dict['2021']['WARD'] = df_dict['2021'].WARD.str.replace('ORL&#201;ANS', 'ORLÉANS')

In [6]:
# There seems to be an issue in some of the ward namings for later years as well.  Here's a quick fix for that
for yr in ['2016','2017','2018','2019','2020','2021']:
    df_dict[yr]['WARD_NUM'] = df_dict[yr].WARD.str.extract('(\d+)').fillna(0).astype(int)
    df_dict[yr] = df_dict[yr].merge(wards[['WARD_NUM','WARD_EN']], on='WARD_NUM', how='left')
    #coalesce WARD_EN with WARD
    df_dict[yr]['WARD'] = df_dict[yr].WARD_EN.combine_first(df_dict[yr].WARD)
    df_dict[yr] = df_dict[yr][['SUBJECT', 'REASON', 'TYPE', 'DATE_RAISED', 'CHANNEL', 'WARD']]

In [7]:
df_all = pd.concat(list(df_dict.values()))

In [8]:
for yr, df in df_dict.items():
    df.to_pickle(f'../data/cleaned/requests_{yr}.pkl')

In [9]:
df_all.to_pickle('../data/cleaned/requests_all.pkl')

In [10]:
wards[['WARD_EN','population']].rename(columns={'WARD_EN':'WARD'}).to_pickle('../data/cleaned/wards.pkl')